In [ ]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

%useLatestDescriptors
%use lets-plot

class Main

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

In [ ]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val vulnCollection = db.getCollection<BsonDocument>("mergedVuln")
val vulnClientCollection = db.getCollection<BsonDocument>("mergedVulnClients")

In [ ]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [ ]:
import org.nield.kotlinstatistics.median

val vulnCveGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val cve = it["cve_ref"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        (cve to gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavToClasses.count())
println(vulnCveGavToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavToClasses.map { it.value.size }.average())
println(vulnCveGavToClasses.map { it.value.size }.median())

In [ ]:
val vulnCveGavPairToClasses = vulnClientCollection.find().useCursor { blk ->
    blk.map {
        val dep_gav = it["dep_gav"]!!.asString().value
        val client_gav = it["client_gav"]!!.asString().value
        val cve = it["cve"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = vulnCveGavToClasses[cve to dep_gav]!!
        
        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        Triple(cve, dep_gav, client_gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavPairToClasses.count())
println(vulnCveGavPairToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavPairToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavPairToClasses.map { it.value.size }.average())
println(vulnCveGavPairToClasses.map { it.value.size }.median())
println(vulnCveGavPairToClasses.keys.map { it.second }.toSet().count()) // are no client for some of the libs in our dataset

In [ ]:
import common.DefaultGraph
import io.github.classgraph.ClassInfoList
import java.io.File
import java.net.URLClassLoader
import org.jgrapht.Graph
import org.jgrapht.graph.DefaultEdge
import org.jgrapht.graph.builder.GraphTypeBuilder

fun loadDepGraph(gav: String): DefaultGraph = 
    scripts.exportDepGraphs.loadDepGraphFromCache(gav)

fun loadClassListInfo(gav: String): ClassInfoList =
    scripts.exportDepGraphs.loadClassInfoListFromCache(gav)

fun loadVertexInfo(gav: String) =
    scripts.exportDepGraphs.loadVertexInfo(gav)

fun loadDepGraph(depGav: String, clientGav: String): DefaultGraph = 
    scripts.exportPairDepGraphs.loadDepGraphFromCache(depGav, clientGav)

fun loadClassListInfo(depGav: String, clientGav: String): ClassInfoList =
    scripts.exportPairDepGraphs.loadClassInfoListFromCache(depGav, clientGav)

fun loadVertexInfo(depGav: String, clientGav: String) =
    scripts.exportPairDepGraphs.loadVertexInfo(depGav, clientGav)

In [ ]:
import com.google.common.collect.Queues

/**
 * returns the depths of each visited node from the starting node
 */
fun bfsOnDepGraph(graph: DefaultGraph, startNode: String): Map<String, Int> {
    val expanded = mutableSetOf<String>()
    val depthMap = mutableMapOf<String, Int>()
    val queue = Queues.newArrayDeque<String>()
    queue.add(startNode)
    depthMap[startNode] = 0
    while (!queue.isEmpty()) {
        val node = queue.pop()
        expanded.add(node)
        val parentDepth = depthMap[node]!! 
        graph
            .outgoingEdgesOf(node)
            .map { e -> graph.getEdgeTarget(e) }
            .filterNot { expanded.contains(it) }
            .forEach { v ->
                if (depthMap.containsKey(v)) {
                    depthMap[v] = min(depthMap[v]!!, parentDepth + 1)
                } else {
                    depthMap[v] = parentDepth + 1;
                }
                queue.add(v)
            }
    }
    return depthMap
}

In [ ]:
import common.DefaultGraph
import io.github.classgraph.ClassGraph
import io.github.classgraph.ClassInfo
import io.github.classgraph.ClassInfoList
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.runBlocking
import org.jgrapht.alg.shortestpath.FloydWarshallShortestPaths
import org.jgrapht.alg.shortestpath.GraphMeasurer
import org.jgrapht.graph.EdgeReversedGraph
import org.jgrapht.traverse.BreadthFirstIterator
import java.lang.reflect.Modifier
import kotlin.Exception

data class VulnAnalysisInfo(
    val depClassCount: Int,
    val depVulnClassCount: Int,
    val clientClassCount: Int,
    val depPublicClassCount: Int,
    val clientPublicClassCount: Int,
    val clientVulnClassCount: Int,
    val clientVulnPublicClassCount: Int,
    val depVulnPublicClassCount: Int,
    val referenedPublicClassCount: Int,
    val referenedVulnPublicClassCount: Int,
) {
    val depVulnRatio = depVulnClassCount.toDouble() / depClassCount.toDouble()
    val depApiSurfaceRatio = depPublicClassCount.toDouble() / depClassCount.toDouble()
    val depVulnApiSurfaceRatio = depVulnPublicClassCount.toDouble() / depPublicClassCount.toDouble()

    val clientVulnRatio = clientVulnClassCount.toDouble() / clientClassCount.toDouble()
    val clientApiSurfaceRatio = clientPublicClassCount.toDouble() / clientClassCount.toDouble()
    val clientVulnApiSurfaceRatio = clientVulnPublicClassCount.toDouble() / clientPublicClassCount.toDouble()
}

System.gc()

@kotlinx.coroutines.ExperimentalCoroutinesApi fun _vulnInfo() =
    Dispatchers.IO.limitedParallelism(32).let { dispatcher ->
        vulnCveGavPairToClasses.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
            runBlocking {
                System.gc()
                println("processing batch $batch")
                w.map {
                    async(dispatcher) {
                        val depGav = it.key.second
                        val clientGav = it.key.third
                        val vulnClasses = it.value

                        val depGraph = loadDepGraph(depGav, clientGav)

                        val vertexInfo = loadVertexInfo(depGav, clientGav)
                        val depVertexInfo = loadVertexInfo(depGav)
                        val clientVertexInfo = vertexInfo.filterNot { depVertexInfo.containsKey(it.key) }.toMap()

                        val depPublicClasses =
                            depGraph
                                .vertexSet()
                                .filter { v -> depVertexInfo.containsKey(v) && Modifier.isPublic(vertexInfo[v]!!) }

                        val publicApiUsages = depGraph.edgeSet().map { e ->
                            val s = depGraph.getEdgeSource(e)
                            val t = depGraph.getEdgeTarget(e)
                            if (clientVertexInfo.keys.contains(s) && depVertexInfo.keys.contains(t)) s to t
                            else null
                        }.filterNotNull()

                        val depPublicClassesUsedByClient = publicApiUsages.map { it.second }.toSet()

                        val vulnDepPublicClassesUsedByClient = depPublicClassesUsedByClient.intersect(vulnClasses)

                        val clientVulnRoots =
                            publicApiUsages
                                .filter { vulnDepPublicClassesUsedByClient.contains(it.second) }
                                .map { it.first }

                        val distToNearestVuln = mutableMapOf<String, Int>()
                        vulnClasses.map {
                            bfsOnDepGraph(EdgeReversedGraph(depGraph), it)
                        }.forEach { m ->
                            m.forEach { v, d ->
                                if (distToNearestVuln.containsKey(v)) {
                                    distToNearestVuln[v] = min(distToNearestVuln[v]!!, d)
                                } else {
                                    distToNearestVuln[v] = d
                                }
                            }
                        }

                        val allVulnClasses = distToNearestVuln.keys.toSet()
                        val allClientVulnClasses = allVulnClasses.intersect(clientVertexInfo.keys)
                        val clientPublicClasses = clientVertexInfo.keys.filter { v ->
                            Modifier.isPublic(clientVertexInfo[v]!!)
                        }
                        val publicClientVulnClasses = allClientVulnClasses.intersect(clientPublicClasses)

                        val depVulnClasses = distToNearestVuln.keys.filter { v ->
                            depVertexInfo.containsKey(v)
                        }.toSet()

                        val depVulnPublicClasses = distToNearestVuln.keys.filter { v ->
                            depVertexInfo.containsKey(v) && Modifier.isPublic(depVertexInfo[v]!!)
                        }.toSet()

                        val res = VulnAnalysisInfo(
                            depClassCount = depVertexInfo.keys.size,
                            depPublicClassCount = depPublicClasses.size,
                            depVulnPublicClassCount = depVulnPublicClasses.size,
                            depVulnClassCount = depVulnClasses.size,
                            clientClassCount = clientVertexInfo.keys.size,
                            clientPublicClassCount = clientPublicClasses.size,
                            clientVulnClassCount = allClientVulnClasses.size,
                            clientVulnPublicClassCount = publicClientVulnClasses.size,
                            referenedPublicClassCount = depPublicClassesUsedByClient.size,
                            referenedVulnPublicClassCount = vulnDepPublicClassesUsedByClient.size,
                        )

                        return@async Result.success(res)
                    }
                }.toList().awaitAll().toList()
            }.toList().filter { it.isSuccess }.map { it.getOrNull()!! }.toList()
        }.toList()
    }

val vulnInfo = _vulnInfo()
vulnInfo.count()

In [ ]:
fun Double.format(digits: Int) =
    "%.${digits}f".format(this)

fun Float.format(digits: Int) =
    "%.${digits}f".format(this)

In [ ]:
println("depClassCount median: ${vulnInfo.map { it.depClassCount }.median()} avg: ${vulnInfo.map { it.depClassCount }.average().format(2)}")
println("depClassCount min: ${vulnInfo.map { it.depClassCount }.min()} max: ${vulnInfo.map { it.depClassCount }.max()}")
println("depPublicClassCount median: ${vulnInfo.map { it.depPublicClassCount }.median()} avg: ${vulnInfo.map { it.depPublicClassCount }.average().format(2)}")
println("depVulnClassCount median: ${vulnInfo.map { it.depVulnClassCount }.median()} avg: ${vulnInfo.map { it.depVulnClassCount }.average().format(2)}")
println("depVulnPublicClassCount median: ${vulnInfo.map { it.depVulnPublicClassCount }.median()} avg: ${vulnInfo.map { it.depVulnPublicClassCount }.average().format(2)}")
println("depApiSurfaceRatio median: ${vulnInfo.map { it.depApiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.map { it.depApiSurfaceRatio * 100 }.average().format(2)}")
println("depVulnApiSurfaceRatio median: ${vulnInfo.map { it.depVulnApiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.map { it.depVulnApiSurfaceRatio * 100 }.average().format(2)}")

In [ ]:
import kotlin.Double.Companion

println("clientClassCount median: ${vulnInfo.map { it.clientClassCount }.median()} avg: ${vulnInfo.map { it.clientClassCount }.average().format(2)}")
println("clientPublicClassCount median: ${vulnInfo.map { it.clientPublicClassCount }.median()} avg: ${vulnInfo.map { it.clientPublicClassCount }.average().format(2)}")
println("clientVulnClassCount median: ${vulnInfo.map { it.clientVulnClassCount }.median()} avg: ${vulnInfo.map { it.clientVulnClassCount }.average().format(2)}")
println("clientVulnPublicClassCount median: ${vulnInfo.map { it.clientVulnPublicClassCount }.median()} avg: ${vulnInfo.map { it.clientVulnPublicClassCount }.average().format(2)}")
println("clientApiSurfaceRatio median: ${vulnInfo.filterNot { it.clientApiSurfaceRatio.isNaN() }.map { it.clientApiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.filterNot { it.clientApiSurfaceRatio.isNaN() }.map { it.clientApiSurfaceRatio * 100 }.average().format(2)}")
println("clientVulnApiSurfaceRatio median: ${vulnInfo.filterNot { it.clientVulnApiSurfaceRatio.isNaN() }.map { it.clientVulnApiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.filterNot { it.clientVulnApiSurfaceRatio.isNaN() }.map { it.clientVulnApiSurfaceRatio * 100 }.average().format(2)}")

In [ ]:
val safeClientCount = vulnInfo.filter { it.clientVulnClassCount == 0 }.count()
val vulnClientCount = vulnInfo.filter { it.clientVulnClassCount > 0 }.count()
println("safe clients: ${safeClientCount} ratio: ${(safeClientCount.toDouble() / vulnInfo.count()).format(2)}")
println("vuln clients: ${vulnClientCount} ratio: ${(vulnClientCount.toDouble() / vulnInfo.count()).format(2)}")

In [ ]:
val nonSafeClientsInfo = vulnInfo.filter { it.clientPublicClassCount > 0 && it.clientVulnClassCount > 0 }
nonSafeClientsInfo.count()

In [ ]:
println("clientClassCount median: ${nonSafeClientsInfo.map { it.clientClassCount }.median()} avg: ${nonSafeClientsInfo.map { it.clientClassCount }.average().format(2)}")
println("clientPublicClassCount median: ${nonSafeClientsInfo.map { it.clientPublicClassCount }.median()} avg: ${nonSafeClientsInfo.map { it.clientPublicClassCount }.average().format(2)}")
println("clientVulnClassCount median: ${nonSafeClientsInfo.map { it.clientVulnClassCount }.median()} avg: ${nonSafeClientsInfo.map { it.clientVulnClassCount }.average().format(2)}")
println("clientVulnPublicClassCount median: ${nonSafeClientsInfo.map { it.clientVulnPublicClassCount }.median()} avg: ${nonSafeClientsInfo.map { it.clientVulnPublicClassCount }.average().format(2)}")
println("clientApiSurfaceRatio median: ${nonSafeClientsInfo.map { it.clientApiSurfaceRatio * 100 }.median().format(2)} avg: ${nonSafeClientsInfo.map { it.clientApiSurfaceRatio * 100 }.average().format(2)}")
println("clientVulnApiSurfaceRatio median: ${nonSafeClientsInfo.map { it.clientVulnApiSurfaceRatio * 100 }.median().format(2)} avg: ${nonSafeClientsInfo.map { it.clientVulnApiSurfaceRatio * 100 }.average().format(2)}")

In [ ]:
import org.jetbrains.kotlinx.dataframe.math.median

println("(vulnInfo) referenedPublicClassCount median: ${vulnInfo.map { it.referenedPublicClassCount }.median()} average: ${vulnInfo.map { it.referenedPublicClassCount }.average()}")
println("referenedPublicClassCount median: ${nonSafeClientsInfo.map { it.referenedPublicClassCount }.median()} average: ${nonSafeClientsInfo.map { it.referenedPublicClassCount }.average()}")
println("referenedVulnPublicClassCount median: ${nonSafeClientsInfo.map { it.referenedVulnPublicClassCount * 100 }.median()} average: ${nonSafeClientsInfo.map { it.referenedVulnPublicClassCount }.average()}")

In [ ]:
val p =
    letsPlot(
        mapOf("clientClassCount" to vulnInfo.filter{it.clientClassCount > 5_000}.map { it.clientClassCount })
    ) { x = "clientClassCount" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 10, center = 5) 
            // xlim(limits = Pair(0, 500))

p

In [ ]:
val p =
    letsPlot(
        mapOf("depClassCount" to vulnInfo.filter{it.depClassCount > 5_000}.map { it.depClassCount })
    ) { x = "depClassCount" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 10, center = 5) 
            // xlim(limits = Pair(0, 500))

p

In [ ]:
ggsave(p, "/tmp/hist.svg")
ggsave(p, "/tmp/hist.png")

In [ ]:
val p =
    letsPlot(
        mapOf("clientClassCount" to vulnInfo.map { it.clientClassCount.coerceAtMost(490) })
    ) { x = "clientClassCount" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 10, center = 5) +
            xlim(limits = Pair(0, 500))

p

In [ ]:
val p =
    letsPlot(
        mapOf("clientApiSurfaceRatio" to vulnInfo.map { it.clientApiSurfaceRatio * 100 })
    ) { x = "clientApiSurfaceRatio" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 2, center = 1) +
            xlim(limits = Pair(0, 100))

p

In [ ]:
val p =
    letsPlot(
        mapOf("clientVulnClassCount" to vulnInfo.map { it.clientVulnClassCount.coerceAtMost(19) })
    ) { x = "clientVulnClassCount" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) +
            xlim(limits = Pair(0, 20))

p

In [ ]:
val p =
    letsPlot(
        mapOf("clientVulnClassCount" to vulnInfo.filter { it.clientVulnClassCount > 0 }.map { it.clientVulnClassCount.coerceAtMost(500) })
    ) { x = "clientVulnClassCount" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) +
            xlim(limits = Pair(0, 501))

p